In [1]:
API_KEY = ""
import os
import openai
import pandas as pd
openai.api_key = API_KEY

In [14]:
import os
import openai
import pandas as pd
openai.api_key = API_KEY

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):

    messages = [{"role": "user", "content": prompt}]

    response = openai.ChatCompletion.create(

    model=model,

    messages=messages,

    temperature=0,

    )
    # print(response)
    return response.choices[0].message["content"]

In [5]:
prompt = "You will work as a Sentiment Analysis for Financial News. You will only answer as: \n\n BEARISH, BULLISH, NEUTRAL. No further explanation=."

question = "This Ford vs. GM Feud Could Shape the Future of EVs in America."

response = get_completion(prompt + question)

print(response)

NEUTRAL


In [6]:
def get_sentiment_completion(question, model="gpt-3.5-turbo"):
    system_analysis_prompt =  "You will work as a Sentiment Analysis for Financial News. You will only answer as: \n\n BEARISH, BULLISH, NEUTRAL. No further explanation=."
    messages = []
    messages = [{"role": "system", "content": system_analysis_prompt}]
    messages.append({"role": "user", "content":question})
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    # print(response)
    return response.choices[0].message["content"]

In [8]:
with open("../data/news_100.txt") as f:
    content = f.readlines()

In [9]:
# nltk_result = []
# for c in content:
#     nltk_result.append(get_sentiment_completion(c))

In [11]:
from tqdm import tqdm
from multiprocess import Pool

max_pool = 5

with Pool(max_pool) as p:
    pool_outputs = list(
        tqdm(
            p.imap(get_sentiment_completion,
                   content),
            total=len(content)
        )
    )    


100%|██████████| 110/110 [00:56<00:00,  1.95it/s]


In [15]:
def convert_output(output):
    converted = []
    dic = {
        'BULLISH': 1,
        'NEUTRAL': 0,
        'BEARISH': -1,
    }
    for i in output:
        if i not in dic:
            print("error")
        else:
            converted.append(dic[i])
    return converted

df_gpt = pd.DataFrame({
    "content": content,
    "sentiment": convert_output(pool_outputs)
})
        
        

In [17]:
df_gpt.to_csv("../results/news_100_gpt.csv")